In [9]:
#Import necessary packages
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder,DeployTransactionBuilder,MessageTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
import requests
import json
import pickle
import csv
import os

from iconsdk.libs.in_memory_zip import gen_deploy_data_content

In [10]:
#GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
SCORE_ADDRESS = "cxda0f8585cc7b40dec53b91a6e12849eac08f266a"
NEWTON_ADDRESS = "hx43e1f6be06a62a293d06867adc27e7acc9affe69"
DEPLOY_PARAMS = {}

In [11]:
#Start ICON service and wallet
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
#icon_service = IconService(HTTPProvider("http://localhost:9000", 3))

# Returns block information by block height
block = icon_service.get_block(1000)
block


{'version': '0.3',
 'height': 1000,
 'signature': 'Oj2VRhXDnd/3IdrE0ClPKEgGLkYLZt/p/o7D9rlspLoSAEA1vYSZyaOoF5n1QV8hpZaObMuZBGaRm13mjYmcHgA=',
 'prev_block_hash': 'cdd0017615d2af92227ec54d3147b199ba1417e1e6ec99cf33e7005a2b709692',
 'merkle_tree_root_hash': '0000000000000000000000000000000000000000000000000000000000000000',
 'time_stamp': 1586159775591512,
 'confirmed_transaction_list': [],
 'block_hash': '95ea963d180936776a650b6b35640c20c1ca951e26a0b55cb11b1b555ecb559b',
 'peer_id': 'hxaad52424d4aec9dac7d9f6796da527f471269d2c',
 'next_leader': 'hxec79e9c1c882632688f8c8f9a07832bcabe8be8f'}

In [14]:
operator = KeyWallet.load('config/yeouido/keystores/operator', 'bgsa199@#')
print(operator.get_address())
print(icon_service.get_balance(operator.get_address()))

hx8b94a3792f336b71937709dae5487166c180c87a
14440018940000000000000


In [ ]:
DEPLOY_PARAMS =  {
            "_tokenName": "TestToken",
            "_symbolName": "TK",
			"_initialSupply": 1000,
            "_decimals": 18
		}
        
deploy_transaction = DeployTransactionBuilder()\
    .from_(operator.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('ODIContracts'))\
    .params(DEPLOY_PARAMS)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, operator, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

In [ ]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

In [43]:
UPDATE_PARAMS =  {
            "_tokenName": "TestToken1",
            "_symbolName": "TK1",
			"_initialSupply": 1000,
            "_decimals": 18
		}
        
deploy_transaction = DeployTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('ODIContracts'))\
    .params(UPDATE_PARAMS)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, operator, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

'0xc0fcd9b9989b95f9bc98be2c5410d6549f44ef7dc394a9a5f92cbfee5342d956'

In [44]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0xc0fcd9b9989b95f9bc98be2c5410d6549f44ef7dc394a9a5f92cbfee5342d956',
 'blockHeight': 3626306,
 'blockHash': '0x64620cf2c2741122f9fbe110be1d34bf654b0043f8ac8af1bf1a83b4a968eb68',
 'txIndex': 1,
 'to': 'cxda0f8585cc7b40dec53b91a6e12849eac08f266a',
 'scoreAddress': 'cxda0f8585cc7b40dec53b91a6e12849eac08f266a',
 'stepUsed': 1756692360,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1756692360,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [45]:
external_methods = ["name", "symbol", "decimals", "totalSupply"]
for method in external_methods:
    call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method(method)\
                    .build()
    print(icon_service.call(call))

TestToken1
TK1
0x12
0x3635c9adc5dea00000


In [47]:
transfer_transaction = CallTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method('transfer')\
    .params({'_to':NEWTON_ADDRESS, '_value':100})\
    .build()

estimate_step = icon_service.estimate_step(transfer_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(transfer_transaction, operator, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

'0x2382a17b221d42b94fe379341ac9e174634d0123403850c3f6051e4750b0e911'

In [49]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0x2382a17b221d42b94fe379341ac9e174634d0123403850c3f6051e4750b0e911',
 'blockHeight': 3626328,
 'blockHash': '0x016e92e9c4f0024ae55292eeb3edebf2555677ff84d35ab4904d32e34a29383e',
 'txIndex': 1,
 'to': 'cxda0f8585cc7b40dec53b91a6e12849eac08f266a',
 'stepUsed': 153880,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 153880,
 'eventLogs': [{'scoreAddress': 'cxda0f8585cc7b40dec53b91a6e12849eac08f266a',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx8b94a3792f336b71937709dae5487166c180c87a',
    'hx43e1f6be06a62a293d06867adc27e7acc9affe69',
    '0x64'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x0

In [51]:
call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method('balanceOf')\
                    .params({'account': NEWTON_ADDRESS})\
                    .build()
print(icon_service.call(call))

0x3635c9adc5dea00000
